In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,TimeSeriesSplit
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from math import sin,log,pow
import lightgbm as lgb
import datetime
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.feature_selection import SelectFromModel, RFECV
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import multiprocessing as mp
import multiprocessing as mp
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None

def importer(path,verbose=True):
  start = datetime.datetime.now()
  df=pd.read_csv(path)
  if verbose:
     print('it took: ', datetime.datetime.now()-start)
  gc.collect()
  return(df)

def pow2(x):
  return pow(x,2)
def log_corr(x):
  return log(1+ abs(x))
def inverse(x):
  return 1/(1+abs(x))
import gc

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)
  
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in tqdm_notebook(df.columns):
        gc.collect()
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):#,missing_correction=True
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    
#     if missing_correction is True:
#         trn_series.astype(object)
#         trn_series.fillna('missing')
#         tst_series.astype(object)
#         tst_series.fillna('missing')

    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [2]:
path='../input/ieee-fraud-detection/'
print('Train readed\n')
train_identity = pd.read_csv(f'{path}train_identity.csv')
train_transaction = pd.read_csv(f'{path}train_transaction.csv')
print('Test readed\n')
test_identity = pd.read_csv(f'{path}test_identity.csv')
test_transaction = pd.read_csv(f'{path}test_transaction.csv')
sub = pd.read_csv(f'{path}sample_submission.csv')
print('Merging\n')
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

del train_identity,train_transaction,test_identity,test_transaction

print('Reducing Memory\n')

train = reduce_mem_usage(train)
test  = reduce_mem_usage(test)

features = test.columns
train = train[features]
train['target'] = 0
test['target'] = 1
train=train.append(test,ignore_index=True)
target = train['target']

train.drop(['target', 'TransactionDT', 'TransactionID'],axis=1,inplace=True)
del test
gc.collect()


Train readed

Test readed

Merging

Reducing Memory




Mem. usage decreased to 650.48 Mb (66.8% reduction)



Mem. usage decreased to 565.37 Mb (66.3% reduction)


0

In [3]:
print('First Checkpoint - Shape : {}\n'.format(train.shape))

Cat= ['ProductCD']+['card'+str(x) for x in range(1,7)]+['addr1','addr2','P_emaildomain','R_emaildomain']+['M'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(12,39)]+['DeviceType','DeviceInfo']
Num=[item for item in train.columns.values.tolist() if item not in Cat]

for f in tqdm_notebook(Cat):
    train[f] = train[f].astype(str)

First Checkpoint - Shape : (1097231, 431)



In [4]:
for f in tqdm_notebook(Cat):
    le = LabelEncoder()
    le.fit(list(train[f].values))
    train[f] = le.transform(list(train[f].values))


In [5]:
print('Replace Infinite Train\n')
for col in tqdm_notebook(train.columns):
    train[col].replace([np.inf, -np.inf], np.nan,inplace=True)

gc.collect()


Replace Infinite Train



0

In [6]:
class Importance_calculator:
    def __init__(self,X,y,param_list,num_round=1000,metric=roc_auc_score,cv=5,random_state=0,modeltype='tree',scale=False):
        
        self.X=X
        self.y=y
        self.cv=cv
        self.scale=scale
        self.param_list=param_list
        self.metric=metric
        self.num_round=num_round
        self.random_state=random_state
        self.modeltype=modeltype
        
    def scorer(self,y_true,y_pred):
        return(self.metric(y_true,y_pred))
  
    def permutate_column_predict(self,model,valid_x,valid_y):
        perm_pred = []
        np.random.seed(self.random_state)
        for col in tqdm_notebook(valid_x.columns):
            value = valid_x[col].copy()
            valid_x[col] = np.random.permutation(valid_x[col].values)
            perm_pred=perm_pred+[self.scorer(valid_y,self.pred_wrapper(model,valid_x,self.modeltype))] #predict
            valid_x[col] = value
        return(perm_pred)
    
    def pred_wrapper(self,model,x,modeltype='logit'):
        if modeltype is 'logit':
            return(model.predict_proba(x)[:,1])
        else:
            return(model.predict(x))
        
    def scaler(self,train,valid):
        train_mean , train_std = train.mean(axis=0),train.std(axis=0)
        #rescale inside the cycle to not overfit
        train-=train_mean
        valid-=train_mean

        train/=train_std
        valid/=train_std
        return(train,valid)
    
    def build_neural(self):
        Input = layers.Input(shape=(self.X.shape[1],))
        x = layer.Dense(self.param_list['number'],activation=self.param_list['activation'])(Input)
        pred=layers.Dense(1,activation='softmax')(x)
        self.NN_model=Model(inputs=Input,outputs=pred)
        
    def cv_score_importance(self):
        N=self.X.shape[1]
        folds = StratifiedKFold(n_splits=self.cv, shuffle=True,random_state=self.random_state)
        print('Inizio train e scoring:\n')
        self.importance_permutation_score=[0]*N
        Error=0
        for trn_idx, val_idx in tqdm_notebook(folds.split(self.X, self.y)):
                train_x, train_y = self.X.iloc[trn_idx], self.y.iloc[trn_idx]
                valid_x, valid_y = self.X.iloc[val_idx], self.y.iloc[val_idx]
                if self.scale is True:
                    train_x,valid_x=self.scaler(train_x,valid_x)
                    
                print('Inizio train.\n')
                if self.modeltype is 'logit':
                    model = LogisticRegression(**self.param_list).fit(train_x, train_y)
                if self.modeltype is 'neural':
                    self.build_neural(param= self.param_list)
                    self.NN_model.compile(optimizer='adam',loss='binary_crossentropy')
                    model = self.NN_model.fit(train_x, train_y,epochs=100)
                if self.modeltype is 'tree':
                    model = lgb.train(self.param_list,lgb.Dataset(train_x, label=train_y),self.num_round)
                
                print('inizio calcolo permutation.\n')
                perm_pred = self.permutate_column_predict(model,valid_x,valid_y)
                Pred=self.scorer(valid_y,self.pred_wrapper(model,valid_x,self.modeltype))
                Error+=Pred
                print('AUC-ROC cv : {}\n'.format(Pred))
                base_pred = [Pred] * N
                tmp_diff=[base_pred[i]-perm_pred[i] for i in range(N)]
                self.importance_permutation_score=[self.importance_permutation_score[i]+tmp_diff[i] for i in range(N)]
        print('AUC-ROC cv: {}\n'.format(Error/self.cv))
        return([self.importance_permutation_score[i]/np.float(self.cv) for i in range(N)])


In [7]:
class Negative_Sampler:
  
  def __init__(self,vector,prob_pos=1,prob_neg=.5,seed=0):
    self.prob_pos=prob_pos
    self.prob_neg=prob_neg
    self.vector=vector
    self.seed=seed
    
  def negative_sample(self):
    np.random.seed(self.seed)
    Positive = np.where(self.vector==1)[0]
    Negative = np.where(self.vector==0)[0]
    Positive_sample= np.random.choice(Positive,np.int(np.round((self.prob_pos)*len(Positive))),replace=False).tolist()
    Negative_sample= np.random.choice(Negative,np.int(np.round((self.prob_neg)*len(Negative))),replace=False).tolist()
    result=np.sort(Positive_sample+Negative_sample)
    return(result)

indexer=Negative_Sampler(vector=target,prob_pos=.5,prob_neg=.5)
index_neg=indexer.negative_sample()
gc.collect()
train_new , target_new = train.iloc[index_neg,:] , target.iloc[index_neg]


In [8]:
gc.collect()

lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'max_bin':255,
                    'verbose':-1,
                } 
imp_model=Importance_calculator(train_new,target_new,lgb_params,cv=5,scale=False)
result=imp_model.cv_score_importance()
del train_new, target_new
gc.collect()
print(result)


Inizio train e scoring:



Inizio train.

inizio calcolo permutation.



AUC-ROC cv : 0.9376648483859147

Inizio train.

inizio calcolo permutation.



AUC-ROC cv : 0.9382674020474878

Inizio train.

inizio calcolo permutation.



AUC-ROC cv : 0.9374875857106691

Inizio train.

inizio calcolo permutation.



AUC-ROC cv : 0.9375367370540778

Inizio train.

inizio calcolo permutation.



AUC-ROC cv : 0.9373227351574726


AUC-ROC cv: 0.9376558616711245

[0.0007132656050678366, 5.913976141402877e-06, 0.0017897422804465713, 0.0018375848480908896, 6.957121140864775e-05, 0.0027489758475235603, 0.0020987363269749705, 0.0007080947643359981, 0.0014263891262092042, 6.20500879379815e-05, 0.05062013619273269, 0.0001533313533460756, 0.0013966343820204009, 1.1691026544458082e-05, 0.0018634858333188785, 0.0010644109409017055, 2.6182801815677338e-05, 1.0268040113770027e-05, 0.0006381325450972808, 0.0012618974976270625, 3.1529366966887015e-05, 1.6342693415705156e-05, 0.01213255187620812, 0.00012447568396478648, 0.008901834651564466, 0.014430646877005926, 0.01258676404958512, 0.001157425706779547, 0.001595202491156833, 0.0012391619947136334, 0.0005848392512540102, 0.015929437524435364, 0.0004143723056729609, 0.00024131913217271262, 5.342208941794268e-06, 0.00011578158773386526, 1.2041254455974482e-05, 0.03844774883062112, 0.03796753100247911, 3.835943810261977e-05, 0.001376614221387928

In [9]:
result_name=pd.DataFrame([result], columns=train.columns)
result_name.to_csv('name.csv', index=False)
print(result_name)

   TransactionAmt  ProductCD    card1     card2    card3     card4     card5  \
0        0.000713   0.000006  0.00179  0.001838  0.00007  0.002749  0.002099   

      card6     addr1     addr2  ...     id_31     id_32     id_33     id_34  \
0  0.000708  0.001426  0.000062  ...  0.071726  0.000037  0.000757  0.004764   

      id_35     id_36     id_37     id_38  DeviceType  DeviceInfo  
0  0.000204 -0.000017  0.001424  0.000584    0.000043    0.000139  

[1 rows x 431 columns]


In [10]:
print(result_name[result_name>=0].dropna(axis=1).columns)



Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=354)
